# Imports necessários

In [246]:
import string
import pandas as pd
from unidecode import unidecode
from fuzzywuzzy import fuzz
import re
from random import randint
import requests
import numpy as np
import os
import csv
import uuid

# Caminho para import dos dados

In [247]:
# Definir uma variável global para armazenar as informações lidas do arquivo
linhas = []

# Definir uma variável de ambiente para armazenar a localização do arquivo
os.environ['PATHS'] = 'despesas.env'

# Função para ler as informações do arquivo e adicioná-las à variável global
def ler_arquivo():
    global linhas
    with open(os.environ.get('PATHS'), 'r') as f:
        linhas = [linha.strip().split(',') for linha in f.readlines()]

# Chamar a função para ler o arquivo e atualizar a variável global
ler_arquivo()

# Imprimir a variável global
print(linhas)

[['C:/Repos/estudos/TCC2/dados/02_despesas/2013/04_analise_tratamento_carga/dados_acumulados_2013.csv'], ['C:/Repos/estudos/TCC2/dados/02_despesas/2014/04_analise_tratamento_carga/dados_acumulados_2014.csv'], ['C:/Repos/estudos/TCC2/dados/02_despesas/2015/04_analise_tratamento_carga/dados_acumulados_2015.csv'], ['C:/Repos/estudos/TCC2/dados/02_despesas/2016/04_analise_tratamento_carga/dados_acumulados_2016.csv'], ['C:/Repos/estudos/TCC2/dados/02_despesas/2017/04_analise_tratamento_carga/dados_acumulados_2017.csv'], ['C:/Repos/estudos/TCC2/dados/02_despesas/2018/04_analise_tratamento_carga/dados_acumulados_2018.csv'], ['C:/Repos/estudos/TCC2/dados/02_despesas/2019/04_analise_tratamento_carga/dados_acumulados_2019.csv'], ['C:/Repos/estudos/TCC2/dados/02_despesas/2020/04_analise_tratamento_carga/dados_acumulados_2020.csv'], ['C:/Repos/estudos/TCC2/dados/02_despesas/2021/04_analise_tratamento_carga/dados_acumulados_2021.csv'], ['C:/Repos/estudos/TCC2/dados/02_despesas/2022/04_analise_trata

## Definindo parâmetros para salvar o arquivo .csv

In [248]:
# definir parâmetros do arquivo CSV
csv_kwargs = {'index': False,
              'sep': ';',
              'quoting': csv.QUOTE_NONE,
              'quotechar': '"'}

## Função para salvar df

In [249]:
def save_dataframe(df, filename):
    # Verifica se o diretório existe e cria caso não exista
    diretorio = './arquivos_para_carga'
    if not os.path.exists(diretorio):
        os.makedirs(diretorio)
    
    # Salva o dataframe em um arquivo csv no diretório criado
    filepath = f'./{diretorio}/{filename}.csv'
    df.to_csv(filepath, index=False, sep=';')

# Despesas

## Importando dados

### Lista com os df

In [250]:
list_data_frames = []

def inserir_df(df):
    list_data_frames.append(df)

In [251]:
df_despesas_2013 = pd.read_csv(linhas[0][0], sep=';')
inserir_df(df_despesas_2013)

df_despesas_2014 = pd.read_csv(linhas[1][0], sep=';')
inserir_df(df_despesas_2014)

df_despesas_2015 = pd.read_csv(linhas[2][0], sep=';')
inserir_df(df_despesas_2015)

df_despesas_2016 = pd.read_csv(linhas[3][0], sep=';')
inserir_df(df_despesas_2016)

df_despesas_2017 = pd.read_csv(linhas[4][0], sep=';')
inserir_df(df_despesas_2017)

df_despesas_2018 = pd.read_csv(linhas[5][0], sep=';')
inserir_df(df_despesas_2018)

df_despesas_2019 = pd.read_csv(linhas[6][0], sep=';')
inserir_df(df_despesas_2019)

df_despesas_2020 = pd.read_csv(linhas[7][0], sep=';')
inserir_df(df_despesas_2020)

df_despesas_2021 = pd.read_csv(linhas[8][0], sep=';')
inserir_df(df_despesas_2021)

df_despesas_2022 = pd.read_csv(linhas[9][0], sep=';')
inserir_df(df_despesas_2022)

### Padronizando os df para junção

In [252]:
dic_renomear_colunas = {'Número':'NUMERO',
                        'Nome':'NOME',
                        'Data':'DATA',
                        'Nome':'CONTRIBUINTE',
                        'Empenhado(R$)':'VALOR',
                        'Liquidado(R$)':'LIQUIDADO',
                        'Pago(R$)': 'PAGO',
                        'Saldo(R$)': 'SALDO',
                        'Tipo':'TIPO',
                        'CODIGO DESPESA': 'COD_DESPESA',
                        'DESPESA':'TIPO_DESPESA',
                        'ORGAO':'CONTRIBUINTE',
                        'DATA INCLUSAO':'DATA_INCLUSAO',
                        'VALOR ARRECADADO':'VALOR',
                        'CODIGO ORGAO':'CODIGO_ORGAO'}

def renomear_colunas(df):
    df.rename(columns=dic_renomear_colunas, inplace=True)
    
for df in list_data_frames:
    renomear_colunas(df)

In [253]:
dados_2020 = ['DATA','CPF/CNPJ','CONTRIBUINTE','VALOR','LIQUIDADO','PAGO','SALDO','TIPO']
dados_2022 = ['DATA','CONTRIBUINTE','VALOR','LIQUIDADO','PAGO','SALDO','TIPO']

df_dados_completos_2020 = pd.DataFrame(columns=dados_2020)
df_dados_completos_2022 = pd.DataFrame(columns=dados_2022)

### Merge Dfs

In [254]:
def merge_dataframes(lista_dataframes, colunas_sem_valores):
    
    df_final = colunas_sem_valores.copy() # cria uma cópia do DataFrame das colunas sem valores
    
    for df in lista_dataframes:
        df_final = pd.merge(df_final, df, on=list(df_final.columns), how='outer') # mescla o DataFrame atual com o DataFrame das colunas
        
    return df_final

In [255]:
def remove_espacos_duplos(df):
    # copia o DataFrame original
    df_copy = df.copy()
    # itera sobre as colunas do DataFrame
    for col in df_copy.columns:
        # verifica se a coluna é do tipo 'object' (string)
        if df_copy[col].dtype == 'O':
            # remove espaços duplicados da coluna
            df_copy[col] = df_copy[col].str.replace('\s+', ' ')
    return df_copy

In [256]:
df_dados_completos_2020 = merge_dataframes(list_data_frames[:8],df_dados_completos_2020)
df_dados_completos_2020.head()

,NUMERO,DATA,CONTRIBUINTE,CPF/CNPJ,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,5,03/01/2013,CAGEPA-COM DE AGUA E ESGOTOS DA PARAIBA,09.123.654/0001-87,45.68,45.68,45.68,0.0,1
1,17,30/01/2013,Prefeitura Municipal de Cajazeiras,08.923.971/0001-15,13309.74,13309.74,13309.74,0.0,1
2,68,30/01/2013,BANCO DO BRASIL S.A.,00.000.000/0099-03,37073.10,37073.10,37073.10,0.0,1
3,69,30/01/2013,BANCO DO BRASIL S.A.,00.000.000/0099-03,3223.75,3223.75,3223.75,0.0,1
4,70,30/01/2013,BANCO DO BRASIL S.A.,00.000.000/0099-03,39698.37,39698.37,39698.37,0.0,1


In [257]:
df_dados_completos_2022 = merge_dataframes(list_data_frames[8:],df_dados_completos_2022)
df_dados_completos_2022.head()

,DATA,CODIGO_ORGAO,CONTRIBUINTE,COD_DESPESA,TIPO_DESPESA,DATA_INCLUSAO,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,31/12/2021,2140,Fundo Municipal de Assistência Social,339039,Outros Serviços de Terceiros - Pessoa Jurídica,12/01/2022,10.45,10.45,10.45,0.0,1
1,31/12/2021,2140,Fundo Municipal de Assistência Social,319004,Contratação por Tempo Determinado,11/01/2022,17820.00,17820.00,17820.00,0.0,1
2,31/12/2021,2140,Fundo Municipal de Assistência Social,319004,Contratação por Tempo Determinado,11/01/2022,24525.03,24525.03,24525.03,0.0,1
3,31/12/2021,2140,Fundo Municipal de Assistência Social,319011,Vencimentos e Vantagens Fixas - Pessoal Civil,11/01/2022,61859.51,61859.51,61859.51,0.0,1
4,31/12/2021,2160,Secretaria Municipal de Desenvolvimento Humano,319011,Vencimentos e Vantagens Fixas - Pessoal Civil,11/01/2022,6500.00,6500.00,6500.00,0.0,1


### Tratamento valores das colunas - Retirando pontuação e acentuação

In [258]:
def tratar_coluna(df, coluna):
    
    # remove sinais de pontuação e símbolos
    table = str.maketrans('', '', string.punctuation)
    df[coluna] = df[coluna].apply(lambda x: str(x).translate(table))
    
    # remove acentuação gráfica
    df[coluna] = df[coluna].apply(lambda x: unidecode(x))
    
    # converte todos os caracteres para minúsculas
    df[coluna] = df[coluna].str.lower()
    
    return df


In [259]:
df_dados_completos_tratados_2020 = tratar_coluna(df_dados_completos_2020, 'CONTRIBUINTE')
df_dados_completos_tratados_2020.head()

,NUMERO,DATA,CONTRIBUINTE,CPF/CNPJ,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,5,03/01/2013,cagepacom de agua e esgotos da paraiba,09.123.654/0001-87,45.68,45.68,45.68,0.0,1
1,17,30/01/2013,prefeitura municipal de cajazeiras,08.923.971/0001-15,13309.74,13309.74,13309.74,0.0,1
2,68,30/01/2013,banco do brasil sa,00.000.000/0099-03,37073.10,37073.10,37073.10,0.0,1
3,69,30/01/2013,banco do brasil sa,00.000.000/0099-03,3223.75,3223.75,3223.75,0.0,1
4,70,30/01/2013,banco do brasil sa,00.000.000/0099-03,39698.37,39698.37,39698.37,0.0,1


In [260]:
df_dados_completos_tratados_2022 = tratar_coluna(df_dados_completos_2022, 'CONTRIBUINTE')
df_dados_completos_tratados_2022 = tratar_coluna(df_dados_completos_2022, 'TIPO_DESPESA')
df_dados_completos_tratados_2022.head()

,DATA,CODIGO_ORGAO,CONTRIBUINTE,COD_DESPESA,TIPO_DESPESA,DATA_INCLUSAO,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,31/12/2021,2140,fundo municipal de assistencia social,339039,outros servicos de terceiros pessoa juridica,12/01/2022,10.45,10.45,10.45,0.0,1
1,31/12/2021,2140,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,17820.00,17820.00,17820.00,0.0,1
2,31/12/2021,2140,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,24525.03,24525.03,24525.03,0.0,1
3,31/12/2021,2140,fundo municipal de assistencia social,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,61859.51,61859.51,61859.51,0.0,1
4,31/12/2021,2160,secretaria municipal de desenvolvimento humano,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,6500.00,6500.00,6500.00,0.0,1


## Criando coluna do Código único de cada contribuinte

### Padronizando dados da coluna contribuinte dos df de 2013 - 2020

In [261]:
dic_padronizar_valores = {' superintendencia cajazeirense de transito':'superintendencia cajazeirense de transito',
                          'fundo municipal de saude':'fundo municipal de saude',
                          ' fundo municipal de saude':'fundo municipal de saude',
                          ' instituto de previdencia e assistencia municipal de cajazeiras':'instituto de previdencia e assistencia municipal de cajazeiras',
                          ' camara municipal':'camara municipal de cajazeiras',
                          'ipam  instituto de previdencia e assistencia municipal de cajazeiras':'instituto de previdencia e assistencia municipal de cajazeiras',
                          'municipio de cajazeiras fundo municipal de saude  fms':'fundo municipal de saude',
                          'ipam  inst prev assist municipal':'instituto de previdencia e assistencia municipal de cajazeiras'}

In [262]:
def padroniza_contribuintes(df, dicionario, coluna):
    # Cria um novo DataFrame para evitar perda de dados do antigo
    novo_df = df.copy()
    
    # Para cada chave no dicionário, encontra as linhas do DataFrame em que o valor na coluna especificada corresponde à chave
    for chave in dicionario:
        novo_df.loc[novo_df[coluna] == chave, coluna] = dicionario[chave]
        
    return novo_df

In [263]:
df_dados_completos_tratados_2020 = padroniza_contribuintes(df_dados_completos_tratados_2020, dic_padronizar_valores, 'CONTRIBUINTE')
df_dados_completos_tratados_2020.head()

,NUMERO,DATA,CONTRIBUINTE,CPF/CNPJ,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,5,03/01/2013,cagepacom de agua e esgotos da paraiba,09.123.654/0001-87,45.68,45.68,45.68,0.0,1
1,17,30/01/2013,prefeitura municipal de cajazeiras,08.923.971/0001-15,13309.74,13309.74,13309.74,0.0,1
2,68,30/01/2013,banco do brasil sa,00.000.000/0099-03,37073.10,37073.10,37073.10,0.0,1
3,69,30/01/2013,banco do brasil sa,00.000.000/0099-03,3223.75,3223.75,3223.75,0.0,1
4,70,30/01/2013,banco do brasil sa,00.000.000/0099-03,39698.37,39698.37,39698.37,0.0,1


In [264]:
df_dados_completos_tratados_2022 = padroniza_contribuintes(df_dados_completos_tratados_2022, dic_padronizar_valores, 'CONTRIBUINTE')
df_dados_completos_tratados_2022.head()

,DATA,CODIGO_ORGAO,CONTRIBUINTE,COD_DESPESA,TIPO_DESPESA,DATA_INCLUSAO,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,31/12/2021,2140,fundo municipal de assistencia social,339039,outros servicos de terceiros pessoa juridica,12/01/2022,10.45,10.45,10.45,0.0,1
1,31/12/2021,2140,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,17820.00,17820.00,17820.00,0.0,1
2,31/12/2021,2140,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,24525.03,24525.03,24525.03,0.0,1
3,31/12/2021,2140,fundo municipal de assistencia social,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,61859.51,61859.51,61859.51,0.0,1
4,31/12/2021,2160,secretaria municipal de desenvolvimento humano,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,6500.00,6500.00,6500.00,0.0,1


### Limpando a coluna contribuinte e criando a coluna CPF/CNPJ dos df de 2021 - 2022

In [265]:
def extrair_cpf_cnpj(df):
    # Expressão regular para identificar sequências de números com mais de 5 dígitos
    regex = r'\d{6,}'
    
    # Criando um novo dataframe para evitar perdas no original
    new_df = df.copy()
    
    # Verifica se a coluna 'CPF/CNPJ' já existe, caso contrário, cria a coluna
    if 'CPF/CNPJ' not in new_df.columns:
        new_df['CPF/CNPJ'] = ''
    
    # Iteração sobre cada linha do novo dataframe
    for i, row in new_df.iterrows():
        # Busca por padrões numéricos na coluna 'CONTRIBUINTE' usando regex
        matches = re.findall(regex, row['CONTRIBUINTE'])
        
        # Se encontrou algum padrão, assume que é um CPF/CNPJ e extrai a primeira ocorrência
        if matches:
            cpf_cnpj = matches[0]
            
            # Remove a sequência de CPF/CNPJ da coluna 'CONTRIBUINTE'
            contribuintes = row['CONTRIBUINTE'].replace(cpf_cnpj, '')
            new_df.at[i, 'CONTRIBUINTE'] = contribuintes
            
            # Armazena o CPF/CNPJ na coluna 'CPF/CNPJ'
            new_df.at[i, 'CPF/CNPJ'] = formatar_cpf_cnpj(cpf_cnpj)
    
    return new_df

def formatar_cpf_cnpj(cpf_cnpj):
    # Verifica o tamanho da string e formata para o tipo correspondente
    if len(cpf_cnpj) == 11:
        cpf_cnpj_formatado = f'{cpf_cnpj[:3]}.{cpf_cnpj[3:6]}.{cpf_cnpj[6:9]}-{cpf_cnpj[9:]}'
    elif len(cpf_cnpj) == 14:
        cpf_cnpj_formatado = f'{cpf_cnpj[:2]}.{cpf_cnpj[2:5]}.{cpf_cnpj[5:8]}/{cpf_cnpj[8:12]}-{cpf_cnpj[12:]}'
    else:
        # Caso a string não seja do tamanho esperado, mantém o valor original
        cpf_cnpj_formatado = cpf_cnpj

    # Retorna a string formatada
    return cpf_cnpj_formatado

In [266]:
df_dados_completos_tratados_2022 = extrair_cpf_cnpj(df_dados_completos_tratados_2022)
df_dados_completos_tratados_2022.head()

,DATA,CODIGO_ORGAO,CONTRIBUINTE,COD_DESPESA,TIPO_DESPESA,DATA_INCLUSAO,VALOR,LIQUIDADO,PAGO,SALDO,TIPO,CPF/CNPJ
0,31/12/2021,2140,fundo municipal de assistencia social,339039,outros servicos de terceiros pessoa juridica,12/01/2022,10.45,10.45,10.45,0.0,1,
1,31/12/2021,2140,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,17820.00,17820.00,17820.00,0.0,1,
2,31/12/2021,2140,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,24525.03,24525.03,24525.03,0.0,1,
3,31/12/2021,2140,fundo municipal de assistencia social,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,61859.51,61859.51,61859.51,0.0,1,
4,31/12/2021,2160,secretaria municipal de desenvolvimento humano,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,6500.00,6500.00,6500.00,0.0,1,


### Preechendo células vazias

In [267]:
def preenche_celulas_vazias(df, column, fill_string):
    # Verifica se a coluna especificada está presente no dataframe
    if column not in df.columns:
        print(f"Coluna {column} não encontrada no dataframe!")
        return df
    
    # Cria uma cópia do dataframe para evitar a perda dos dados originais
    df_novo = df.copy()

    # Preenche células vazias na coluna 'CPF/CNPJ' com a mensagem 'nao ha registro do documento original'
    df_novo[column].fillna(fill_string, inplace=True)

    # Preenche células vazias na coluna 'CPF/CNPJ' com a mensagem 'nao ha registro do documento original'
    df_novo[column].replace('', fill_string, inplace=True)

    return df_novo

In [268]:
df_dados_completos_tratados_2022 = preenche_celulas_vazias(df_dados_completos_tratados_2022,'CPF/CNPJ','sem informacao')
df_dados_completos_tratados_2022.head()

,DATA,CODIGO_ORGAO,CONTRIBUINTE,COD_DESPESA,TIPO_DESPESA,DATA_INCLUSAO,VALOR,LIQUIDADO,PAGO,SALDO,TIPO,CPF/CNPJ
0,31/12/2021,2140,fundo municipal de assistencia social,339039,outros servicos de terceiros pessoa juridica,12/01/2022,10.45,10.45,10.45,0.0,1,sem informacao
1,31/12/2021,2140,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,17820.00,17820.00,17820.00,0.0,1,sem informacao
2,31/12/2021,2140,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,24525.03,24525.03,24525.03,0.0,1,sem informacao
3,31/12/2021,2140,fundo municipal de assistencia social,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,61859.51,61859.51,61859.51,0.0,1,sem informacao
4,31/12/2021,2160,secretaria municipal de desenvolvimento humano,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,6500.00,6500.00,6500.00,0.0,1,sem informacao


In [269]:
df_dados_completos_tratados_2020 = preenche_celulas_vazias(df_dados_completos_tratados_2020,'CPF/CNPJ','sem informacao')
df_dados_completos_tratados_2020.head()

,NUMERO,DATA,CONTRIBUINTE,CPF/CNPJ,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,5,03/01/2013,cagepacom de agua e esgotos da paraiba,09.123.654/0001-87,45.68,45.68,45.68,0.0,1
1,17,30/01/2013,prefeitura municipal de cajazeiras,08.923.971/0001-15,13309.74,13309.74,13309.74,0.0,1
2,68,30/01/2013,banco do brasil sa,00.000.000/0099-03,37073.10,37073.10,37073.10,0.0,1
3,69,30/01/2013,banco do brasil sa,00.000.000/0099-03,3223.75,3223.75,3223.75,0.0,1
4,70,30/01/2013,banco do brasil sa,00.000.000/0099-03,39698.37,39698.37,39698.37,0.0,1


### Criação da coluna única do código

In [270]:
def df_to_dict(df, coluna_chave, coluna_valor,coluna_agrupadora):
    """
    Converte duas colunas de um DataFrame em um dicionário.

    Args:
        df (pandas.DataFrame): DataFrame contendo as colunas a serem convertidas.
        coluna_chave (str): Nome da coluna a ser usada como chave do dicionário.
        coluna_valor (str): Nome da coluna a ser usada como valor do dicionário.
        coluna_agrupadora (str): Nome da coluna a ser usada como coluna agrupadora.

    Returns:
        dict: Dicionário com os valores das colunas especificadas.
    """

    # Cria um dicionário vazio
    dicionario = {}
    
    # Agrupa os dados por tipo
    grupos = df.groupby(coluna_agrupadora)

    # Percorre cada grupo e adiciona a chave e valor correspondentes no dicionário
    for nome_grupo, grupo in grupos:
        # Cria um dicionário para o grupo atual
        dict_grupo = {}
        # Percorre cada linha do grupo e adiciona a chave e valor correspondentes no dicionário do grupo
        for indice, linha in grupo.iterrows():
            chave = linha[coluna_chave]
            valor = linha[coluna_valor]
            dict_grupo[chave] = valor
        # Adiciona o dicionário do grupo ao dicionário geral, usando o nome do grupo como chave
        dicionario[nome_grupo] = dict_grupo

    return dicionario

In [271]:
def merge_dataframes(df1, df2):
    
    # Cria cópias dos dataframes para evitar perda de dados
    df1_copy = df1.copy()
    df2_copy = df2.copy()
    
    # Itera sobre cada linha do primeiro dataframe
    for i, row1 in df1_copy.iterrows():
        string_comparacao = row1['CONTRIBUINTE'] + '-' + str(row1['TIPO'])
        
        # Itera sobre cada linha do segundo dataframe
        for j, row2 in df2_copy.iterrows():
            
            # Compara o valor da coluna 'CONTRIBUINTE' em ambas as linhas
            if string_comparacao == row2['CONCAT_CODIGO']:
                
                # Atribui o valor da coluna 'CPF/CNPJ' do primeiro dataframe no segundo
                df2_copy.at[j, 'CPF/CNPJ'] = row1['CPF/CNPJ']
    
    # Faz o merge dos dois dataframes sem valores duplicados
    merged_df = pd.concat([df1_copy, df2_copy]).reset_index(drop=True)
    
    return merged_df

In [272]:
import hashlib

def gerar_codigo_unico(df, coluna_parametro):
    # Cria uma cópia do dataframe apenas com as colunas necessárias para a codificação
    df_copy = df[[coluna_parametro]].copy()

    # Cria um dicionário vazio para armazenar os códigos únicos gerados
    codigos = {}

    # Percorre cada linha do dataframe e armazena o código único gerado a partir do valor da coluna_parametro
    # no dicionário de códigos, se ainda não estiver lá
    for index, row in df_copy.iterrows():
        valor = row[coluna_parametro]
        if valor not in codigos:
            # Calcula o hash MD5 do valor da coluna_parametro
            codigo_hash = hashlib.md5(str(valor).encode('utf-8')).hexdigest()

            # Pega os primeiros 14 caracteres do hash como código único
            codigo_unico = codigo_hash[:14]

            # Armazena o código único no dicionário, com a chave sendo o valor da coluna_parametro
            codigos[valor] = codigo_unico
    
    # Cria uma nova coluna CODIGO no dataframe e preenche com os códigos gerados
    df['CODIGO'] = [codigos[row[coluna_parametro]] for index, row in df_copy.iterrows()]

    # Retorna o dataframe com a nova coluna CODIGO
    return df

In [273]:
df_dados_completos_tratados_2020.head()

,NUMERO,DATA,CONTRIBUINTE,CPF/CNPJ,VALOR,LIQUIDADO,PAGO,SALDO,TIPO
0,5,03/01/2013,cagepacom de agua e esgotos da paraiba,09.123.654/0001-87,45.68,45.68,45.68,0.0,1
1,17,30/01/2013,prefeitura municipal de cajazeiras,08.923.971/0001-15,13309.74,13309.74,13309.74,0.0,1
2,68,30/01/2013,banco do brasil sa,00.000.000/0099-03,37073.10,37073.10,37073.10,0.0,1
3,69,30/01/2013,banco do brasil sa,00.000.000/0099-03,3223.75,3223.75,3223.75,0.0,1
4,70,30/01/2013,banco do brasil sa,00.000.000/0099-03,39698.37,39698.37,39698.37,0.0,1


In [274]:
dic_valores_2020 = df_to_dict(df_dados_completos_2020,'CPF/CNPJ','TIPO','TIPO')
dic_valores_2020

{1: {'09.123.654/0001-87': 1,
  '08.923.971/0001-15': 1,
  '00.000.000/0099-03': 1,
  '09.283.185/0001-63': 1,
  '738.558.394-68': 1,
  '30.822.936/0001-69': 1,
  '00.000.000/0001-91': 1,
  '10.609.902/0001-83': 1,
  '09.238.207/0001-73': 1,
  '10.293.888/0001-51': 1,
  '752.937.064-20': 1,
  '07.748.243/0001-51': 1,
  '33.000.118/0012-21': 1,
  '10.334.550/0001-09': 1,
  '330.968.994-87': 1,
  '40.973.307/0001-93': 1,
  '840.958.814-53': 1,
  '797.248.504-63': 1,
  '893.455.334-00': 1,
  '08.605.487/0001-48': 1,
  '00.000.313/0001-40': 1,
  '079.762.514-30': 1,
  '692.076.404-06': 1,
  '646.556.104-25': 1,
  '33.669.672/0070-75': 1,
  '09.095.183/0001-40': 1,
  '00.394.460/0086-30': 1,
  '09.087.693/0001-76': 1,
  '06.292.667/0001-91': 1,
  '05.323.998/0001-89': 1,
  '024.122.264-82': 1,
  '018.584.044-28': 1,
  '10.954.450/0001-77': 1,
  '11.663.900/0001-35': 1,
  '29.979.036/0169-00': 1,
  '00.703.157/0001-83': 1,
  '04.941.967/0001-29': 1,
  '09.353.327/0001-11': 1,
  '12.724.464/0

In [275]:
df_dados_completos_tratados_2020['CONCAT_CODIGO'] = df_dados_completos_tratados_2020.apply(lambda x: x['CPF/CNPJ'] + '-' + str(x['TIPO']), axis=1)
df_dados_completos_tratados_2020.head()

,NUMERO,DATA,CONTRIBUINTE,CPF/CNPJ,VALOR,LIQUIDADO,PAGO,SALDO,TIPO,CONCAT_CODIGO
0,5,03/01/2013,cagepacom de agua e esgotos da paraiba,09.123.654/0001-87,45.68,45.68,45.68,0.0,1,09.123.654/0001-87-1
1,17,30/01/2013,prefeitura municipal de cajazeiras,08.923.971/0001-15,13309.74,13309.74,13309.74,0.0,1,08.923.971/0001-15-1
2,68,30/01/2013,banco do brasil sa,00.000.000/0099-03,37073.10,37073.10,37073.10,0.0,1,00.000.000/0099-03-1
3,69,30/01/2013,banco do brasil sa,00.000.000/0099-03,3223.75,3223.75,3223.75,0.0,1,00.000.000/0099-03-1
4,70,30/01/2013,banco do brasil sa,00.000.000/0099-03,39698.37,39698.37,39698.37,0.0,1,00.000.000/0099-03-1


In [276]:
df_contribuintes_receitas_2020 = pd.DataFrame(df_dados_completos_tratados_2020[['CPF/CNPJ','CONTRIBUINTE','TIPO','CONCAT_CODIGO']])
df_contribuintes_receitas_2020.drop_duplicates(subset='CONCAT_CODIGO', inplace=True)
df_contribuintes_receitas_2020.reset_index(drop=True)
df_contribuintes_receitas_2020[['TIPO']].value_counts()


TIPO
1       5546
3        394
2        344
4          4
dtype: int64

In [277]:
def remove_espacos_brancos_inicio(df):
    for coluna in df.columns:
        if df[coluna].dtype == 'object':
            df[coluna] = df[coluna].str.lstrip()
    return df


In [278]:
df_dados_completos_tratados_2022 = remove_espacos_brancos_inicio(df_dados_completos_tratados_2022)

In [279]:
df_dados_completos_tratados_2022.head()

,DATA,CODIGO_ORGAO,CONTRIBUINTE,COD_DESPESA,TIPO_DESPESA,DATA_INCLUSAO,VALOR,LIQUIDADO,PAGO,SALDO,TIPO,CPF/CNPJ
0,31/12/2021,2140,fundo municipal de assistencia social,339039,outros servicos de terceiros pessoa juridica,12/01/2022,10.45,10.45,10.45,0.0,1,sem informacao
1,31/12/2021,2140,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,17820.00,17820.00,17820.00,0.0,1,sem informacao
2,31/12/2021,2140,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,24525.03,24525.03,24525.03,0.0,1,sem informacao
3,31/12/2021,2140,fundo municipal de assistencia social,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,61859.51,61859.51,61859.51,0.0,1,sem informacao
4,31/12/2021,2160,secretaria municipal de desenvolvimento humano,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,6500.00,6500.00,6500.00,0.0,1,sem informacao


In [280]:
dic_valores_2022 = df_to_dict(df_dados_completos_2022,'CONTRIBUINTE','TIPO','TIPO')
dic_valores_2022

{1: {' fundo municipal de assistencia social': 1,
  ' secretaria municipal de desenvolvimento humano': 1,
  ' fundo municipal de saude': 1,
  ' secretaria municipal de educacao': 1,
  ' secretaria municipal de juventude esporte e turismo': 1,
  ' procuradoria geral do municipio': 1,
  ' secretaria municipal da fazenda publica': 1,
  ' secretaria municipal de saude': 1,
  ' secretaria de governo e articulacao politica': 1,
  ' secretaria municipal de politicas publicas': 1,
  ' secretaria municipal de politicas publicas para as mulheres': 1,
  ' secretaria municipal de cultura e turismo': 1,
  ' secretaria municipal de controle social': 1,
  ' secretaria municipal do desenvolvimento rural e do meio ambiente': 1,
  ' secretaria municipal de infraestrutura': 1,
  ' secretaria municipal de administracao': 1,
  ' secretaria municipal de planejamento': 1,
  ' secretaria municipal de comunicacao': 1,
  ' secretaria municipal de meio ambiente': 1,
  ' secretaria municipal de juventude e esport

In [281]:
df_dados_completos_tratados_2022['CONCAT_CODIGO'] = df_dados_completos_tratados_2022.apply(lambda x: x['CONTRIBUINTE'] + '-' + str(x['TIPO']), axis=1)
df_dados_completos_tratados_2022.head()

,DATA,CODIGO_ORGAO,CONTRIBUINTE,COD_DESPESA,TIPO_DESPESA,DATA_INCLUSAO,VALOR,LIQUIDADO,PAGO,SALDO,TIPO,CPF/CNPJ,CONCAT_CODIGO
0,31/12/2021,2140,fundo municipal de assistencia social,339039,outros servicos de terceiros pessoa juridica,12/01/2022,10.45,10.45,10.45,0.0,1,sem informacao,fundo municipal de assistencia social-1
1,31/12/2021,2140,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,17820.00,17820.00,17820.00,0.0,1,sem informacao,fundo municipal de assistencia social-1
2,31/12/2021,2140,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,24525.03,24525.03,24525.03,0.0,1,sem informacao,fundo municipal de assistencia social-1
3,31/12/2021,2140,fundo municipal de assistencia social,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,61859.51,61859.51,61859.51,0.0,1,sem informacao,fundo municipal de assistencia social-1
4,31/12/2021,2160,secretaria municipal de desenvolvimento humano,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,6500.00,6500.00,6500.00,0.0,1,sem informacao,secretaria municipal de desenvolvimento humano-1


In [282]:
df_contribuintes_receitas_2022 = pd.DataFrame(df_dados_completos_tratados_2022[['CPF/CNPJ','CONTRIBUINTE','TIPO','CONCAT_CODIGO']])
df_contribuintes_receitas_2022.drop_duplicates(subset='CONCAT_CODIGO', inplace=True)
df_contribuintes_receitas_2022.reset_index(drop=True)
df_contribuintes_receitas_2022[['TIPO']].value_counts()


TIPO
1       21
2        1
3        1
4        1
dtype: int64

## Merge dos Df

### Merge dos dados dos contribuintes

In [283]:
df_contribuinte_merged = merge_dataframes(df_contribuintes_receitas_2020, df_contribuintes_receitas_2022)
df_contribuinte_merged

,CPF/CNPJ,CONTRIBUINTE,TIPO,CONCAT_CODIGO
0,09.123.654/0001-87,cagepacom de agua e esgotos da paraiba,1,09.123.654/0001-87-1
1,08.923.971/0001-15,prefeitura municipal de cajazeiras,1,08.923.971/0001-15-1
2,00.000.000/0099-03,banco do brasil sa,1,00.000.000/0099-03-1
3,09.283.185/0001-63,poder judiciario do estado da paraiba,1,09.283.185/0001-63-1
4,738.558.394-68,henrique sergio alves da cunha,1,738.558.394-68-1
...,...,...,...,...
6307,08.841.553/0001-89,camara municipal de cajazeiras,4,camara municipal de cajazeiras-4
6308,sem informacao,secretaria municipal de comunicacao,1,secretaria municipal de comunicacao-1
6309,sem informacao,secretaria municipal de meio ambiente,1,secretaria municipal de meio ambiente-1
6310,sem informacao,secretaria municipal de juventude e esporte,1,secretaria municipal de juventude e esporte-1


In [284]:
df_contribuinte_merged[['TIPO']].value_counts()

TIPO
1       5567
3        395
2        345
4          5
dtype: int64

In [285]:
df_contribuinte_merged = gerar_codigo_unico(df_contribuinte_merged,'CONCAT_CODIGO')
df_contribuinte_merged.sort_values(['TIPO'], ascending=True, inplace=True)
df_contribuinte_merged.reset_index(drop=True)

,CPF/CNPJ,CONTRIBUINTE,TIPO,CONCAT_CODIGO,CODIGO
0,09.123.654/0001-87,cagepacom de agua e esgotos da paraiba,1,09.123.654/0001-87-1,86eabac73f48c4
1,080.329.514-66,raniely duarte barreto,1,080.329.514-66-1,c86a88163a502f
2,025.419.924-08,cicera de freitas rocha,1,025.419.924-08-1,0d5215bcd76d11
3,526.406.044-49,maria rivonia pereira de oliveira,1,526.406.044-49-1,6709a10d3ece82
4,823.237.713-53,luciana barreto rolim,1,823.237.713-53-1,a4c224f5451c8e
...,...,...,...,...,...
6307,102.097.424-96,prefeitura municipal de cajazeiras,4,102.097.424-96-4,2521c02e3c6b10
6308,08.923.971/0001-15,prefeitura municipal de cajazeiras,4,08.923.971/0001-15-4,6e8245200e6967
6309,646.603.624-34,marcos helder,4,646.603.624-34-4,a50e69b97e2b34
6310,08.841.553/0001-89,camara municipal de cajazeiras,4,08.841.553/0001-89-4,24bf0d009e9239


In [286]:
def remove_espacos_duplos(df):
    # copia o DataFrame original
    df_copy = df.copy()
    # itera sobre as colunas do DataFrame
    for col in df_copy.columns:
        # verifica se a coluna é do tipo 'object' (string)
        if df_copy[col].dtype == 'O':
            # remove espaços duplicados da coluna
            df_copy[col] = df_copy[col].str.replace('\s+', ' ')
    return df_copy

In [287]:
df_contribuinte_merged = remove_espacos_duplos(df_contribuinte_merged)

C:\Users\MacInBox\AppData\Local\Temp\ipykernel_4576\2630336717.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df_copy[col] = df_copy[col].str.replace('\s+', ' ')


In [288]:
save_dataframe(df_contribuinte_merged,'dados_contribuintes_despesas')

with open('arquivos_para_carga/dados_contribuintes_despesas.csv', 'w', encoding='utf-8', newline='') as f:
    df_contribuinte_merged.to_csv(f, **csv_kwargs)

## Dados Tipos de Despesa

In [289]:
df_dados_completos_tratados_2022.head()

,DATA,CODIGO_ORGAO,CONTRIBUINTE,COD_DESPESA,TIPO_DESPESA,DATA_INCLUSAO,VALOR,LIQUIDADO,PAGO,SALDO,TIPO,CPF/CNPJ,CONCAT_CODIGO
0,31/12/2021,2140,fundo municipal de assistencia social,339039,outros servicos de terceiros pessoa juridica,12/01/2022,10.45,10.45,10.45,0.0,1,sem informacao,fundo municipal de assistencia social-1
1,31/12/2021,2140,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,17820.00,17820.00,17820.00,0.0,1,sem informacao,fundo municipal de assistencia social-1
2,31/12/2021,2140,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,24525.03,24525.03,24525.03,0.0,1,sem informacao,fundo municipal de assistencia social-1
3,31/12/2021,2140,fundo municipal de assistencia social,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,61859.51,61859.51,61859.51,0.0,1,sem informacao,fundo municipal de assistencia social-1
4,31/12/2021,2160,secretaria municipal de desenvolvimento humano,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,6500.00,6500.00,6500.00,0.0,1,sem informacao,secretaria municipal de desenvolvimento humano-1


In [290]:
df_tipos_despesa = pd.DataFrame(df_dados_completos_tratados_2022[['COD_DESPESA','TIPO_DESPESA']])
df_tipos_despesa.head()

,COD_DESPESA,TIPO_DESPESA
0,339039,outros servicos de terceiros pessoa juridica
1,319004,contratacao por tempo determinado
2,319004,contratacao por tempo determinado
3,319011,vencimentos e vantagens fixas pessoal civil
4,319011,vencimentos e vantagens fixas pessoal civil


In [291]:
df_tipos_despesa.drop_duplicates(subset='COD_DESPESA', inplace=True)
df_tipos_despesa.reset_index(drop=True, inplace=True)


In [292]:
df_tipos_despesa = remove_espacos_duplos(df_tipos_despesa)
df_tipos_despesa

C:\Users\MacInBox\AppData\Local\Temp\ipykernel_4576\2630336717.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df_copy[col] = df_copy[col].str.replace('\s+', ' ')


,COD_DESPESA,TIPO_DESPESA
0,339039,outros servicos de terceiros pessoa juridica
1,319004,contratacao por tempo determinado
2,319011,vencimentos e vantagens fixas pessoal civil
3,339030,material de consumo
4,335041,contribuicoes
5,339036,outros servicos de terceiros pessoa fisica
6,339091,sentencas judiciais
7,339048,outros auxilios financeiros a pessoas fisicas
8,469071,principal da divida contratual resgatado
9,319113,contribuicoes patronais


In [293]:
df_tipos_despesa.loc[len(df_tipos_despesa)] = ['000001','nao ha dados sobre a classe dessa despesa']

In [294]:
df_tipos_despesa

,COD_DESPESA,TIPO_DESPESA
0,339039,outros servicos de terceiros pessoa juridica
1,319004,contratacao por tempo determinado
2,319011,vencimentos e vantagens fixas pessoal civil
3,339030,material de consumo
4,335041,contribuicoes
5,339036,outros servicos de terceiros pessoa fisica
6,339091,sentencas judiciais
7,339048,outros auxilios financeiros a pessoas fisicas
8,469071,principal da divida contratual resgatado
9,319113,contribuicoes patronais


In [295]:
save_dataframe(df_tipos_despesa,'dados_tipo_despesas')

with open('arquivos_para_carga/dados_tipo_despesas.csv', 'w', encoding='utf-8', newline='') as f:
    df_tipos_despesa.to_csv(f, **csv_kwargs)

## Dados Despesa

In [296]:
df_contribuinte = merge_dataframes(df_contribuintes_receitas_2020, df_contribuintes_receitas_2022)
df_contribuinte = gerar_codigo_unico(df_contribuinte,'CONCAT_CODIGO')
df_contribuinte.sort_values(['TIPO'], ascending=True, inplace=True)
df_contribuinte_codigo = pd.DataFrame(df_contribuinte[['CONCAT_CODIGO', 'CODIGO']])
df_contribuinte_codigo.reset_index(drop=True)

,CONCAT_CODIGO,CODIGO
0,09.123.654/0001-87-1,86eabac73f48c4
1,080.329.514-66-1,c86a88163a502f
2,025.419.924-08-1,0d5215bcd76d11
3,526.406.044-49-1,6709a10d3ece82
4,823.237.713-53-1,a4c224f5451c8e
...,...,...
6307,102.097.424-96-4,2521c02e3c6b10
6308,08.923.971/0001-15-4,6e8245200e6967
6309,646.603.624-34-4,a50e69b97e2b34
6310,08.841.553/0001-89-4,24bf0d009e9239


In [297]:
df_dados_completos_tratados_2020.head()

,NUMERO,DATA,CONTRIBUINTE,CPF/CNPJ,VALOR,LIQUIDADO,PAGO,SALDO,TIPO,CONCAT_CODIGO
0,5,03/01/2013,cagepacom de agua e esgotos da paraiba,09.123.654/0001-87,45.68,45.68,45.68,0.0,1,09.123.654/0001-87-1
1,17,30/01/2013,prefeitura municipal de cajazeiras,08.923.971/0001-15,13309.74,13309.74,13309.74,0.0,1,08.923.971/0001-15-1
2,68,30/01/2013,banco do brasil sa,00.000.000/0099-03,37073.10,37073.10,37073.10,0.0,1,00.000.000/0099-03-1
3,69,30/01/2013,banco do brasil sa,00.000.000/0099-03,3223.75,3223.75,3223.75,0.0,1,00.000.000/0099-03-1
4,70,30/01/2013,banco do brasil sa,00.000.000/0099-03,39698.37,39698.37,39698.37,0.0,1,00.000.000/0099-03-1


In [298]:
df_dados_2020=pd.merge(df_dados_completos_tratados_2020, df_contribuinte_codigo, on='CONCAT_CODIGO', how='left')
df_dados_completos_2020 = pd.DataFrame(df_dados_2020[['DATA','VALOR','LIQUIDADO','PAGO','SALDO','TIPO','CODIGO']])
df_dados_completos_2020.rename({'TIPO':'FONTE_DADOS','CODIGO':'COD_CONTRIBUINTE'}, axis=1, inplace=True)
df_dados_completos_2020 = df_dados_completos_2020.reindex(['COD_DESPESA','DATA','COD_CONTRIBUINTE','VALOR','LIQUIDADO','PAGO','SALDO','FONTE_DADOS'], axis=1)
df_dados_completos_2020['DATA'] = pd.to_datetime(df_dados_completos_2020['DATA'], format='%d/%m/%Y')
df_dados_completos_2020['COD_DESPESA'].fillna('000001', inplace=True)

df_dados_completos_2020.head()

,COD_DESPESA,DATA,COD_CONTRIBUINTE,VALOR,LIQUIDADO,PAGO,SALDO,FONTE_DADOS
0,000001,2013-01-03,86eabac73f48c4,45.68,45.68,45.68,0.0,1
1,000001,2013-01-30,886cd0eabf5a18,13309.74,13309.74,13309.74,0.0,1
2,000001,2013-01-30,2d027177369c9d,37073.10,37073.10,37073.10,0.0,1
3,000001,2013-01-30,2d027177369c9d,3223.75,3223.75,3223.75,0.0,1
4,000001,2013-01-30,2d027177369c9d,39698.37,39698.37,39698.37,0.0,1


In [299]:
df_dados_completos_tratados_2022.head()

,DATA,CODIGO_ORGAO,CONTRIBUINTE,COD_DESPESA,TIPO_DESPESA,DATA_INCLUSAO,VALOR,LIQUIDADO,PAGO,SALDO,TIPO,CPF/CNPJ,CONCAT_CODIGO
0,31/12/2021,2140,fundo municipal de assistencia social,339039,outros servicos de terceiros pessoa juridica,12/01/2022,10.45,10.45,10.45,0.0,1,sem informacao,fundo municipal de assistencia social-1
1,31/12/2021,2140,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,17820.00,17820.00,17820.00,0.0,1,sem informacao,fundo municipal de assistencia social-1
2,31/12/2021,2140,fundo municipal de assistencia social,319004,contratacao por tempo determinado,11/01/2022,24525.03,24525.03,24525.03,0.0,1,sem informacao,fundo municipal de assistencia social-1
3,31/12/2021,2140,fundo municipal de assistencia social,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,61859.51,61859.51,61859.51,0.0,1,sem informacao,fundo municipal de assistencia social-1
4,31/12/2021,2160,secretaria municipal de desenvolvimento humano,319011,vencimentos e vantagens fixas pessoal civil,11/01/2022,6500.00,6500.00,6500.00,0.0,1,sem informacao,secretaria municipal de desenvolvimento humano-1


In [313]:
df_dados_2022=pd.merge(df_dados_completos_tratados_2022, df_contribuinte_codigo, on='CONCAT_CODIGO', how='left')
df_dados_completos_2022 = pd.DataFrame(df_dados_2022[['COD_DESPESA','DATA','VALOR','LIQUIDADO','PAGO','SALDO','TIPO','CODIGO']])
df_dados_completos_2022.rename({'TIPO':'FONTE_DADOS','CODIGO':'COD_CONTRIBUINTE'}, axis=1, inplace=True)
df_dados_completos_2022 = df_dados_completos_2022.reindex(['COD_DESPESA','DATA','COD_CONTRIBUINTE','VALOR','LIQUIDADO','PAGO','SALDO','FONTE_DADOS'], axis=1)
df_dados_completos_2022['DATA'] = pd.to_datetime(df_dados_completos_2022['DATA'], format='%d/%m/%Y')
df_dados_completos_2022['COD_DESPESA'] = df_dados_completos_2022['COD_DESPESA'].astype(str)


df_dados_completos_2022.head()


,COD_DESPESA,DATA,COD_CONTRIBUINTE,VALOR,LIQUIDADO,PAGO,SALDO,FONTE_DADOS
0,339039,2021-12-31,1a6d83df959c2f,10.45,10.45,10.45,0.0,1
1,319004,2021-12-31,1a6d83df959c2f,17820.00,17820.00,17820.00,0.0,1
2,319004,2021-12-31,1a6d83df959c2f,24525.03,24525.03,24525.03,0.0,1
3,319011,2021-12-31,1a6d83df959c2f,61859.51,61859.51,61859.51,0.0,1
4,319011,2021-12-31,960bbd4fb54364,6500.00,6500.00,6500.00,0.0,1


In [314]:
df_dados_completos_2022['COD_DESPESA'].dtype

dtype('O')

In [315]:
df_receitas = pd.concat([df_dados_completos_2022,df_dados_completos_2020])
df_receitas
df_receitas.isnull().value_counts()


COD_DESPESA  DATA   COD_CONTRIBUINTE  VALOR  LIQUIDADO  PAGO   SALDO  FONTE_DADOS
False        False  False             False  False      False  False  False          88316
dtype: int64

In [316]:
df_receitas = remove_espacos_duplos(df_receitas)
df_receitas.head()

C:\Users\MacInBox\AppData\Local\Temp\ipykernel_4576\2630336717.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df_copy[col] = df_copy[col].str.replace('\s+', ' ')


,COD_DESPESA,DATA,COD_CONTRIBUINTE,VALOR,LIQUIDADO,PAGO,SALDO,FONTE_DADOS
0,339039,2021-12-31,1a6d83df959c2f,10.45,10.45,10.45,0.0,1
1,319004,2021-12-31,1a6d83df959c2f,17820.00,17820.00,17820.00,0.0,1
2,319004,2021-12-31,1a6d83df959c2f,24525.03,24525.03,24525.03,0.0,1
3,319011,2021-12-31,1a6d83df959c2f,61859.51,61859.51,61859.51,0.0,1
4,319011,2021-12-31,960bbd4fb54364,6500.00,6500.00,6500.00,0.0,1


In [317]:
save_dataframe(df_receitas,'dados_despesas')

with open('arquivos_para_carga/dados_despesas.csv', 'w', encoding='utf-8', newline='') as f:
    df_receitas.to_csv(f, **csv_kwargs)